# Machine Learning in Julia (continued)

An introduction to the
[MLJ](https://alan-turing-institute.github.io/MLJ.jl/stable/)
toolbox.

### Set-up

Inspect Julia version:

In [1]:
VERSION

v"1.7.1"

The following instantiates a package environment.

The package environment has been created using **Julia 1.6** and may not
instantiate properly for other Julia versions.

In [2]:
using Pkg
Pkg.activate("env")
Pkg.instantiate()

  Activating project at `~/GoogleDrive/Julia/MLJ/MLJTutorial/notebooks/03_pipelines/env`


## General resources

- [MLJ Cheatsheet](https://alan-turing-institute.github.io/MLJ.jl/dev/mlj_cheatsheet/)
- [Common MLJ Workflows](https://alan-turing-institute.github.io/MLJ.jl/dev/common_mlj_workflows/)
- [MLJ manual](https://alan-turing-institute.github.io/MLJ.jl/dev/)
- [Data Science Tutorials in Julia](https://juliaai.github.io/DataScienceTutorials.jl/)

## Part 3 - Transformers and Pipelines

### Transformers

Unsupervised models, which receive no target `y` during training,
always have a `transform` operation. They sometimes also support an
`inverse_transform` operation, with obvious meaning, and sometimes
support a `predict` operation (see the clustering example discussed
[here](https://alan-turing-institute.github.io/MLJ.jl/dev/transformers/#Transformers-that-also-predict-1)).
Otherwise, they are handled much like supervised models.

Here's a simple standardization example:

In [3]:
using MLJ

x = rand(100);
@show mean(x) std(x);

mean(x) = 0.47253618410343046
std(x) = 0.3027701918985175


In [4]:
model = Standardizer() # a built-in model
mach = machine(model, x)
fit!(mach)
xhat = transform(mach, x);
@show mean(xhat) std(xhat);

[ Info: Training Machine{Standardizer,…}.
mean(xhat) = 5.773159728050814e-17
std(xhat) = 1.0


This particular model has an `inverse_transform`:

In [5]:
inverse_transform(mach, xhat) ≈ x

true

### Re-encoding the King County House data as continuous

For further illustrations of transformers, let's re-encode *all* of the
King County House input features (see [Ex
3](#exercise-3-fixing-scitypes-in-a-table)) into a set of `Continuous`
features. We do this with the `ContinuousEncoder` model, which, by
default, will:

- one-hot encode all `Multiclass` features
- coerce all `OrderedFactor` features to `Continuous` ones
- coerce all `Count` features to `Continuous` ones (there aren't any)
- drop any remaining non-Continuous features (none of these either)

First, we reload the data and fix the scitypes (Exercise 3):

In [6]:
using UrlDownload, CSV
import DataFrames
house_csv = urldownload("https://raw.githubusercontent.com/ablaom/"*
                        "MachineLearningInJulia2020/for-MLJ-version-0.16/"*
                        "data/house.csv");
house = DataFrames.DataFrame(house_csv)
coerce!(house, autotype(house_csv));
coerce!(house, Count => Continuous, :zipcode => Multiclass);
schema(house)

┌───────────────┬───────────────────┬───────────────────────────────────┐
│ names         │ scitypes          │ types                             │
├───────────────┼───────────────────┼───────────────────────────────────┤
│ price         │ Continuous        │ Float64                           │
│ bedrooms      │ OrderedFactor{13} │ CategoricalValue{Int64, UInt32}   │
│ bathrooms     │ OrderedFactor{30} │ CategoricalValue{Float64, UInt32} │
│ sqft_living   │ Continuous        │ Float64                           │
│ sqft_lot      │ Continuous        │ Float64                           │
│ floors        │ OrderedFactor{6}  │ CategoricalValue{Float64, UInt32} │
│ waterfront    │ OrderedFactor{2}  │ CategoricalValue{Int64, UInt32}   │
│ view          │ OrderedFactor{5}  │ CategoricalValue{Int64, UInt32}   │
│ condition     │ OrderedFactor{5}  │ CategoricalValue{Int64, UInt32}   │
│ grade         │ OrderedFactor{12} │ CategoricalValue{Int64, UInt32}   │
│ sqft_above    │ Continuous        │ 

In [7]:
y, X = unpack(house, ==(:price), rng=123);

Instantiate the unsupervised model (transformer):

In [8]:
encoder = ContinuousEncoder() # a built-in model; no need to @load it

ContinuousEncoder(
    drop_last = false,
    one_hot_ordered_factors = false)

Bind the model to the data and fit!

In [9]:
mach = machine(encoder, X) |> fit!;

[ Info: Training Machine{ContinuousEncoder,…}.


Transform and inspect the result:

In [10]:
Xcont = transform(mach, X);
schema(Xcont)

┌────────────────┬────────────┬─────────┐
│ names          │ scitypes   │ types   │
├────────────────┼────────────┼─────────┤
│ bedrooms       │ Continuous │ Float64 │
│ bathrooms      │ Continuous │ Float64 │
│ sqft_living    │ Continuous │ Float64 │
│ sqft_lot       │ Continuous │ Float64 │
│ floors         │ Continuous │ Float64 │
│ waterfront     │ Continuous │ Float64 │
│ view           │ Continuous │ Float64 │
│ condition      │ Continuous │ Float64 │
│ grade          │ Continuous │ Float64 │
│ sqft_above     │ Continuous │ Float64 │
│ sqft_basement  │ Continuous │ Float64 │
│ yr_built       │ Continuous │ Float64 │
│ zipcode__98001 │ Continuous │ Float64 │
│ zipcode__98002 │ Continuous │ Float64 │
│ zipcode__98003 │ Continuous │ Float64 │
│ zipcode__98004 │ Continuous │ Float64 │
│       ⋮        │     ⋮      │    ⋮    │
└────────────────┴────────────┴─────────┘
                          71 rows omitted


### More transformers

Here's how to list all of MLJ's unsupervised models:

In [11]:
models(m->!m.is_supervised)

60-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ABODDetector, package_name = OutlierDetectionNeighbors, ... )
 (name = ABODDetector, package_name = OutlierDetectionPython, ... )
 (name = AEDetector, package_name = OutlierDetectionNetworks, ... )
 (name = AffinityPropagation, package_name = ScikitLearn, ... )
 (name = AgglomerativeClustering, package_name = ScikitLearn, ... )
 (name = BM25Transformer, package_name = MLJText, ... )
 (name = BagOfWordsTransformer, package_name = ML

Some commonly used ones are built-in (do not require `@load`ing):

model type                  | does what?
----------------------------|----------------------------------------------
ContinuousEncoder | transform input table to a table of `Continuous` features (see above)
FeatureSelector | retain or dump selected features
FillImputer | impute missing values
OneHotEncoder | one-hot encoder `Multiclass` (and optionally `OrderedFactor`) features
Standardizer | standardize (whiten) a vector or all `Continuous` features of a table
UnivariateBoxCoxTransformer | apply a learned Box-Cox transformation to a vector
UnivariateDiscretizer | discretize a `Continuous` vector, and hence render its elscitypw `OrderedFactor`

In addition to "dynamic" transformers (ones that learn something
from the data and must be `fit!`) users can wrap ordinary functions
as transformers, and such *static* transformers can depend on
parameters, like the dynamic ones. See
[here](https://alan-turing-institute.github.io/MLJ.jl/dev/transformers/#Static-transformers-1)
for how to define your own static transformers.

### Pipelines

In [12]:
length(schema(Xcont).names)

87

Let's suppose that additionally we'd like to reduce the dimension of
our data.  A model that will do this is `PCA` from
`MultivariateStats.jl`:

In [13]:
PCA = @load PCA
reducer = PCA()

[ Info: For silent loading, specify `verbosity=0`. 
import MLJMultivariateStatsInterface ✔


PCA(
    maxoutdim = 0,
    method = :auto,
    pratio = 0.99,
    mean = nothing)

Now, rather simply repeating the work-flow above, applying the new
transformation to `Xcont`, we can combine both the encoding and the
dimension-reducing models into a single model, known as a
*pipeline*. While MLJ offers a powerful interface for composing
models in a variety of ways, we'll stick to these simplest class of
composite models for now. The simplest way to construct a pipeline
is using the Julia's `|>` syntax:

In [14]:
pipe = encoder |> reducer

UnsupervisedPipeline(
    continuous_encoder = ContinuousEncoder(
            drop_last = false,
            one_hot_ordered_factors = false),
    pca = PCA(
            maxoutdim = 0,
            method = :auto,
            pratio = 0.99,
            mean = nothing),
    cache = true)

Notice that the model `pipe` has other models as hyperparameters
(with names automatically generated based on the mode type
name). The hyperparameters of the component models are are now
*nested*, but we can still access them:

In [15]:
@show pipe.pca.pratio
pipe.pca.pratio = 0.85

pipe.pca.pratio = 0.99


0.85

The pipeline model behaves like any other transformer:

In [16]:
mach = machine(pipe, X)
fit!(mach)
Xsmall = transform(mach, X)
schema(Xsmall)

[ Info: Training Machine{UnsupervisedPipeline{NamedTuple{,…},…},…}.
[ Info: Training Machine{ContinuousEncoder,…}.
[ Info: Training Machine{PCA,…}.


┌───────┬────────────┬─────────┐
│ names │ scitypes   │ types   │
├───────┼────────────┼─────────┤
│ x1    │ Continuous │ Float64 │
└───────┴────────────┴─────────┘


Want to combine this pre-processing with ridge regression?

In [17]:
RidgeRegressor = @load RidgeRegressor pkg=MLJLinearModels
rgs = RidgeRegressor()
pipe2 = pipe |> rgs

[ Info: For silent loading, specify `verbosity=0`. 
import MLJLinearModels ✔


DeterministicPipeline(
    continuous_encoder = ContinuousEncoder(
            drop_last = false,
            one_hot_ordered_factors = false),
    pca = PCA(
            maxoutdim = 0,
            method = :auto,
            pratio = 0.85,
            mean = nothing),
    ridge_regressor = RidgeRegressor(
            lambda = 1.0,
            fit_intercept = true,
            penalize_intercept = false,
            solver = nothing),
    cache = true)

Now our pipeline is a supervised model, instead of a transformer,
whose performance we can evaluate:

In [18]:
mach = machine(pipe2, X, y)
evaluate!(mach, measure=mae, resampling=Holdout()) # CV(nfolds=6) is default

PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────┬─────────────┬───────────┬────────────┐
│ measure             │ measurement │ operation │ per_fold   │
├─────────────────────┼─────────────┼───────────┼────────────┤
│ MeanAbsoluteError() │ 234000.0    │ predict   │ [234000.0] │
└─────────────────────┴─────────────┴───────────┴────────────┘


### Training of composite models is "smart"

Now notice what happens if we train on all the data, then change a
regressor hyper-parameter and retrain:

In [19]:
fit!(mach)

[ Info: Training Machine{DeterministicPipeline{NamedTuple{,…},…},…}.
[ Info: Training Machine{ContinuousEncoder,…}.
[ Info: Training Machine{PCA,…}.
[ Info: Training Machine{RidgeRegressor,…}.


Machine{DeterministicPipeline{NamedTuple{,…},…},…} trained 2 times; caches data
  model: MLJBase.DeterministicPipeline{NamedTuple{(:continuous_encoder, :pca, :ridge_regressor), Tuple{MLJModelInterface.Unsupervised, MLJModelInterface.Unsupervised, MLJModelInterface.Deterministic}}, MLJModelInterface.predict}
  args: 
    1:	Source @864 ⏎ `ScientificTypesBase.Table{Union{AbstractVector{ScientificTypesBase.Continuous}, AbstractVector{ScientificTypesBase.Multiclass{70}}, AbstractVector{ScientificTypesBase.OrderedFactor{6}}, AbstractVector{ScientificTypesBase.OrderedFactor{13}}, AbstractVector{ScientificTypesBase.OrderedFactor{30}}, AbstractVector{ScientificTypesBase.OrderedFactor{5}}, AbstractVector{ScientificTypesBase.OrderedFactor{12}}, AbstractVector{ScientificTypesBase.OrderedFactor{2}}}}`
    2:	Source @134 ⏎ `AbstractVector{ScientificTypesBase.Continuous}`


In [20]:
pipe2.ridge_regressor.lambda = 0.1
fit!(mach)

[ Info: Updating Machine{DeterministicPipeline{NamedTuple{,…},…},…}.
[ Info: Not retraining Machine{ContinuousEncoder,…}. Use `force=true` to force.
[ Info: Not retraining Machine{PCA,…}. Use `force=true` to force.
[ Info: Updating Machine{RidgeRegressor,…}.


Machine{DeterministicPipeline{NamedTuple{,…},…},…} trained 3 times; caches data
  model: MLJBase.DeterministicPipeline{NamedTuple{(:continuous_encoder, :pca, :ridge_regressor), Tuple{MLJModelInterface.Unsupervised, MLJModelInterface.Unsupervised, MLJModelInterface.Deterministic}}, MLJModelInterface.predict}
  args: 
    1:	Source @864 ⏎ `ScientificTypesBase.Table{Union{AbstractVector{ScientificTypesBase.Continuous}, AbstractVector{ScientificTypesBase.Multiclass{70}}, AbstractVector{ScientificTypesBase.OrderedFactor{6}}, AbstractVector{ScientificTypesBase.OrderedFactor{13}}, AbstractVector{ScientificTypesBase.OrderedFactor{30}}, AbstractVector{ScientificTypesBase.OrderedFactor{5}}, AbstractVector{ScientificTypesBase.OrderedFactor{12}}, AbstractVector{ScientificTypesBase.OrderedFactor{2}}}}`
    2:	Source @134 ⏎ `AbstractVector{ScientificTypesBase.Continuous}`


Second time only the ridge regressor is retrained!

Mutate a hyper-parameter of the `PCA` model and every model except
the `ContinuousEncoder` (which comes before it will be retrained):

In [21]:
pipe2.pca.pratio = 0.9999
fit!(mach)

[ Info: Updating Machine{DeterministicPipeline{NamedTuple{,…},…},…}.
[ Info: Not retraining Machine{ContinuousEncoder,…}. Use `force=true` to force.
[ Info: Updating Machine{PCA,…}.
[ Info: Training Machine{RidgeRegressor,…}.


Machine{DeterministicPipeline{NamedTuple{,…},…},…} trained 4 times; caches data
  model: MLJBase.DeterministicPipeline{NamedTuple{(:continuous_encoder, :pca, :ridge_regressor), Tuple{MLJModelInterface.Unsupervised, MLJModelInterface.Unsupervised, MLJModelInterface.Deterministic}}, MLJModelInterface.predict}
  args: 
    1:	Source @864 ⏎ `ScientificTypesBase.Table{Union{AbstractVector{ScientificTypesBase.Continuous}, AbstractVector{ScientificTypesBase.Multiclass{70}}, AbstractVector{ScientificTypesBase.OrderedFactor{6}}, AbstractVector{ScientificTypesBase.OrderedFactor{13}}, AbstractVector{ScientificTypesBase.OrderedFactor{30}}, AbstractVector{ScientificTypesBase.OrderedFactor{5}}, AbstractVector{ScientificTypesBase.OrderedFactor{12}}, AbstractVector{ScientificTypesBase.OrderedFactor{2}}}}`
    2:	Source @134 ⏎ `AbstractVector{ScientificTypesBase.Continuous}`


### Inspecting composite models

The dot syntax used above to change the values of *nested*
hyper-parameters is also useful when inspecting the learned
parameters and report generated when training a composite model:

In [22]:
fitted_params(mach).ridge_regressor

(coefs = [:x1 => -0.7328956348956909, :x2 => -0.16590563202915437, :x3 => 194.59515890822126, :x4 => 102.71301756136401],
 intercept = 540085.6428739978,)

In [23]:
report(mach).pca

(indim = 87,
 outdim = 4,
 tprincipalvar = 2.463215246230867e9,
 tresidualvar = 157533.26199626923,
 tvar = 2.463372779492863e9,
 mean = [4.369869985656781, 8.45912182482765, 2079.8997362698374, 15106.967565816869, 1.988617961412113, 1.0075417572757137, 1.2343034284921113, 3.4094295100171195, 6.6569194466293435, 1788.3906907879516  …  0.011798454633785222, 0.012122333780595013, 0.006292509138018785, 0.012955165872391617, 0.01466709850552908, 47.560052519317075, -122.21389640494147, 1986.552491556008, 12768.455651691113, 1.9577106371165502],
 principalvars = [2.177071551045086e9, 2.8418139726430327e8, 1.685016083064339e6, 277281.8384131848],)

### Incorporating target transformations

Next, suppose that instead of using the raw `:price` as the training
target, we want to use the log-price (a common practice in dealing
with house price data). However, suppose that we still want to
report final *predictions* on the original linear scale (and use
these for evaluation purposes). Then we wrap our supervised model
using `TransformedTargetModel`, which has to key-word arguments
`target` and `inverse`.

First we'll overload `log` and `exp` for broadcasting:

In [24]:
Base.log(v::AbstractArray) = log.(v)
Base.exp(v::AbstractArray) = exp.(v)

Now for the new pipeline:

In [25]:
rgs_log = TransformedTargetModel(rgs, target=log, inverse=exp)

pipe3 = pipe |> rgs_log
mach = machine(pipe3, X, y)
evaluate!(mach, measure=mae)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:06


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────┬─────────────┬───────────┬───────────────────────────────
│ measure             │ measurement │ operation │ per_fold                     ⋯
├─────────────────────┼─────────────┼───────────┼───────────────────────────────
│ MeanAbsoluteError() │ 162000.0    │ predict   │ [160000.0, 161000.0, 164000. ⋯
└─────────────────────┴─────────────┴───────────┴───────────────────────────────
                                                                1 column omitted


MLJ will also allow you to insert *learned* target
transformations. For example, we might want to apply
`Standardizer()` to the target, to standardize it, or
`UnivariateBoxCoxTransformer()` to make it look Gaussian. Then
instead of specifying a *function* for `target`, we specify a
unsupervised *model* (or model type). One does not specify `inverse`
because only models implementing `inverse_transform` are
allowed.

Let's see which of these two options results in a better outcome:

In [26]:
box = UnivariateBoxCoxTransformer(n=20)
stand = Standardizer()

rgs_box = TransformedTargetModel(rgs, target=box)
pipe4 = pipe |> rgs_box
mach = machine(pipe4, X, y)
evaluate!(mach, measure=mae)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:15


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────┬─────────────┬───────────┬───────────────────────────────
│ measure             │ measurement │ operation │ per_fold                     ⋯
├─────────────────────┼─────────────┼───────────┼───────────────────────────────
│ MeanAbsoluteError() │ 479000.0    │ predict   │ [168000.0, 172000.0, 170000. ⋯
└─────────────────────┴─────────────┴───────────┴───────────────────────────────
                                                                1 column omitted


In [27]:
pipe4.transformed_target_model_deterministic.target = stand
evaluate!(mach, measure=mae)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:03


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────┬─────────────┬───────────┬───────────────────────────────
│ measure             │ measurement │ operation │ per_fold                     ⋯
├─────────────────────┼─────────────┼───────────┼───────────────────────────────
│ MeanAbsoluteError() │ 172000.0    │ predict   │ [173000.0, 171000.0, 172000. ⋯
└─────────────────────┴─────────────┴───────────┴───────────────────────────────
                                                                1 column omitted


### Resources for Part 3

- From the MLJ manual:
    - [Transformers and other unsupervised models](https://alan-turing-institute.github.io/MLJ.jl/dev/transformers/)
    - [Linear pipelines](https://alan-turing-institute.github.io/MLJ.jl/dev/linear_pipelines/#Linear-Pipelines)
- From Data Science Tutorials:
    - [Composing models](https://juliaai.github.io/DataScienceTutorials.jl/getting-started/composing-models/)

### Exercises for Part 3

#### Exercise 7

Consider again the Horse Colic classification problem considered in
Exercise 6, but with all features, `Finite` and `Infinite`:

In [28]:
csv_file = urldownload("https://raw.githubusercontent.com/ablaom/"*
                   "MachineLearningInJulia2020/"*
                   "for-MLJ-version-0.16/data/horse.csv");
horse = DataFrames.DataFrame(csv_file); # convert to data frame
coerce!(horse, autotype(horse));
coerce!(horse, Count => Continuous);
coerce!(horse,
        :surgery               => Multiclass,
        :age                   => Multiclass,
        :mucous_membranes      => Multiclass,
        :capillary_refill_time => Multiclass,
        :outcome               => Multiclass,
        :cp_data               => Multiclass);

y, X = unpack(horse, ==(:outcome));
schema(X)

┌─────────────────────────┬──────────────────┬──────────────────────────────────
│ names                   │ scitypes         │ types                           ⋯
├─────────────────────────┼──────────────────┼──────────────────────────────────
│ surgery                 │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} ⋯
│ age                     │ Multiclass{2}    │ CategoricalValue{Int64, UInt32} ⋯
│ rectal_temperature      │ Continuous       │ Float64                         ⋯
│ pulse                   │ Continuous       │ Float64                         ⋯
│ respiratory_rate        │ Continuous       │ Float64                         ⋯
│ temperature_extremities │ OrderedFactor{4} │ CategoricalValue{Int64, UInt32} ⋯
│ mucous_membranes        │ Multiclass{6}    │ CategoricalValue{Int64, UInt32} ⋯
│ capillary_refill_time   │ Multiclass{3}    │ CategoricalValue{Int64, UInt32} ⋯
│ pain                    │ OrderedFactor{5} │ CategoricalValue{Int64, UInt32} ⋯
│ peristalsis             │ 

(a) Define a pipeline that:
- uses `Standardizer` to ensure that features that are already
  continuous are centered at zero and have unit variance
- re-encodes the full set of features as `Continuous`, using
  `ContinuousEncoder`
- uses the `KMeans` clustering model from `Clustering.jl`
  to reduce the dimension of the feature space to `k=10`.
- trains a `EvoTreeClassifier` (a gradient tree boosting
  algorithm in `EvoTrees.jl`) on the reduced data, using
  `nrounds=50` and default values for the other
   hyper-parameters

(b) Evaluate the pipeline on all data, using 6-fold cross-validation
and `cross_entropy` loss.

&star;(c) Plot a learning curve which examines the effect on this loss
as the tree booster parameter `max_depth` varies from 2 to 10.

<a id='part-4-tuning-hyper-parameters'></a>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*